In [ ]:
import pandas as pd

from evidently.v2.metrics import Metric
from evidently.v2.datasets import DataDefinition
from evidently.v2.datasets import Dataset
from evidently.v2.metrics import SingleValue
from evidently.v2.metrics import SingleValueCheck

In [ ]:
from evidently.descriptors import TextLength

from evidently.v2.datasets import FeatureScorer

data = pd.DataFrame(data={"column_1": [1, 2, 3, 4, -1, 5], "column_2": ["a", "aa", "aaaa", "aaaaaaa", "a", "aa"]})

dataset = Dataset.from_pandas(
    data,
    data_definition=None,
    scorers={
        "text_length": FeatureScorer(TextLength().feature("column_2")),
    },
)

In [ ]:
dataset.as_dataframe()

In [ ]:
from evidently.descriptors import TextLength

from evidently.v2.checks.numerical_checks import le, ge
from typing import Optional
from typing import List
from plotly.express import line

# user-friendly interface
def max_metric(column_name: str, checks: Optional[List[SingleValueCheck]] = None, group_by: Optional[str] = None) -> Metric:
    return MaxMetric(column_name, checks)


# implementation
class MaxMetric(Metric[SingleValue]):
    _column_name: str
    
    def __init__(self, column_name: str, checks: Optional[List[SingleValueCheck]] = None):
        super().__init__(f"max:{column_name}")
        self._column_name = column_name
        self._checks = checks if checks is not None else [le(10), ge(6)]

    def calculate(self, current_data: Dataset, reference_data: Optional[Dataset]) -> SingleValue:
        x = current_data.column(self._column_name).data
        value = x.max()
        result = SingleValue(value=value)
        figure = line(x)
        figure.add_hrect(6, 10)
        #result.set_widget([plotly_figure(title=self.display_name(), figure=figure)])
        return result

    def display_name(self) -> str:
        return f"Max value for {self._column_name}"

# usage example
data = pd.DataFrame(data={"column_1": [1, 2, 3, 4, -1, 5], "column_2": ["a", "aa", "aaaa", "aaaaaaa", "a", "aa"]})

dataset = Dataset.from_pandas(data, data_definition=None, scorers={
    "text_length": FeatureScorer(TextLength().feature("column_2")),
})
result = max_metric("column_1", checks=None).call(dataset, None)
result

In [ ]:
from evidently.v2.metrics.group_by import GroupBy
from evidently.v2.metrics.base import render_results
from evidently.v2.report import Context

context = Context()

context.init_dataset(dataset, None)

metrics = GroupBy(max_metric("column_1"), "column_2").generate_metrics(context)

results = [metric.call(dataset, None) for metric in metrics]

render_results(results)